# Bash trap command

---

It often comes the situations that you want to catch a special signal/interruption/user input in your script to prevent the unpredictables.

Trap is your command to try:

- `trap <arg/function> <signal>`

Some of the common signal types you can trap:

- `SIGINT`: user sends an interrupt signal (Ctrl + C)
- `SIGQUIT`: user sends a quit signal (Ctrl + D)
- `SIGFPE`: attempted an illegal mathematical operation

You can check out all signal types by entering the following command:

In [1]:
kill -l

 1) SIGHUP	 2) SIGINT	 3) SIGQUIT	 4) SIGILL	 5) SIGTRAP
 6) SIGABRT	 7) SIGBUS	 8) SIGFPE	 9) SIGKILL	10) SIGUSR1
11) SIGSEGV	12) SIGUSR2	13) SIGPIPE	14) SIGALRM	15) SIGTERM
16) SIGSTKFLT	17) SIGCHLD	18) SIGCONT	19) SIGSTOP	20) SIGTSTP
21) SIGTTIN	22) SIGTTOU	23) SIGURG	24) SIGXCPU	25) SIGXFSZ
26) SIGVTALRM	27) SIGPROF	28) SIGWINCH	29) SIGIO	30) SIGPWR
31) SIGSYS	34) SIGRTMIN	35) SIGRTMIN+1	36) SIGRTMIN+2	37) SIGRTMIN+3
38) SIGRTMIN+4	39) SIGRTMIN+5	40) SIGRTMIN+6	41) SIGRTMIN+7	42) SIGRTMIN+8
43) SIGRTMIN+9	44) SIGRTMIN+10	45) SIGRTMIN+11	46) SIGRTMIN+12	47) SIGRTMIN+13
48) SIGRTMIN+14	49) SIGRTMIN+15	50) SIGRTMAX-14	51) SIGRTMAX-13	52) SIGRTMAX-12
53) SIGRTMAX-11	54) SIGRTMAX-10	55) SIGRTMAX-9	56) SIGRTMAX-8	57) SIGRTMAX-7
58) SIGRTMAX-6	59) SIGRTMAX-5	60) SIGRTMAX-4	61) SIGRTMAX-3	62) SIGRTMAX-2
63) SIGRTMAX-1	64) SIGRTMAX	
 6) SIGABRT	 7) SIGBUS	 8) SIGFPE	 9) SIGKILL	10) SIGUSR1
11) SIGSEGV	12) SIGUSR2	13) SIGPIPE	14) SIGALRM	15) SIGTERM
16) SIGSTKFLT	17) SIGCHLD	18) SIGCONT	19) 

Notice the numbers before each signal name, you can use that number to avoid typing long strings in trap:

```bash
#2 corresponds to SIGINT and 15 corresponds to SIGTERM
trap booh 2 15
```

One of the common usage of trap is to do cleanup temporary files:

```bash
trap "rm -f folder; exit" 2
```

## Exercise 1: Basic Cleanup

Create a script with cleanup on exit:

In [2]:
cat > /tmp/cleanup_demo.sh << 'EOF'
#!/bin/bash
cleanup() {
    echo "Cleaning up..."
    rm -f /tmp/test$$
}
trap cleanup EXIT

echo "Working..." > /tmp/test$$
cat /tmp/test$$
echo "Done"
EOF

bash /tmp/cleanup_demo.sh

Working...
Done
Cleaning up...
Done
Cleaning up...


## Exercise 2: Handle Interrupts

Create a script that handles Ctrl+C gracefully (Note: Ctrl+C won't work in notebooks, but shows the concept):

In [3]:
cat > /tmp/interrupt_handler.sh << 'EOF'
#!/bin/bash
interrupted() {
    echo "Script interrupted!"
    exit 1
}
trap interrupted SIGINT

# Simulate some work
for i in {1..3}; do
    echo "Running... iteration $i"
    sleep 1
done
echo "Completed successfully"
EOF

bash /tmp/interrupt_handler.sh

Running... iteration 1
Running... iteration 2
Running... iteration 2
Running... iteration 3
Running... iteration 3
Completed successfully
Completed successfully


## Exercise 3: Multiple Signals

Handle different signals with different actions:

In [4]:
cat > /tmp/multi_signal.sh << 'EOF'
#!/bin/bash
handle_int() { echo "Got SIGINT"; }
handle_term() { echo "Got SIGTERM"; exit; }

trap handle_int SIGINT
trap handle_term SIGTERM

echo "Script running with PID: $$"
echo "Send signals using: kill -SIGINT $$ or kill -SIGTERM $$"

# Simulate work
for i in {1..3}; do
    echo "Working... $i"
    sleep 1
done
EOF

bash /tmp/multi_signal.sh

Script running with PID: 212749
Send signals using: kill -SIGINT 212749 or kill -SIGTERM 212749
Working... 1
Send signals using: kill -SIGINT 212749 or kill -SIGTERM 212749
Working... 1
Working... 2
Working... 2
Working... 3
Working... 3


## Exercise 4: Temp File Cleanup

Create a script that always cleans up temporary files:

In [5]:
cat > /tmp/temp_cleanup.sh << 'EOF'
#!/bin/bash
TMPFILE=$(mktemp /tmp/demo.XXXXXX)
cleanup() {
    echo "Removing $TMPFILE"
    rm -f "$TMPFILE"
}
trap cleanup EXIT SIGINT SIGTERM

echo "Using temp file: $TMPFILE"
echo "data" > "$TMPFILE"
cat "$TMPFILE"
echo "Work completed"
EOF

bash /tmp/temp_cleanup.sh

Using temp file: /tmp/demo.xdfqJw
data
Work completed
Removing /tmp/demo.xdfqJw
data
Work completed
Removing /tmp/demo.xdfqJw


## Practical Example: Database Backup Script

Here's a practical example showing trap usage in a backup scenario:

In [6]:
cat > /tmp/backup_script.sh << 'EOF'
#!/bin/bash

BACKUP_DIR="/tmp/backup_$$"
LOCK_FILE="/tmp/backup.lock"

cleanup() {
    echo "Cleaning up..."
    rm -f "$LOCK_FILE"
    rm -rf "$BACKUP_DIR"
    echo "Cleanup complete"
}

# Set trap for various exit scenarios
trap cleanup EXIT
trap "echo 'Interrupted!'; exit 1" INT TERM

# Create lock file
if [ -f "$LOCK_FILE" ]; then
    echo "Backup already running!"
    exit 1
fi

touch "$LOCK_FILE"
mkdir -p "$BACKUP_DIR"

echo "Starting backup to $BACKUP_DIR"
echo "This is a simulated backup" > "$BACKUP_DIR/data.txt"
sleep 1
echo "Backup complete"
EOF

bash /tmp/backup_script.sh

Starting backup to /tmp/backup_212803
Backup complete
Cleaning up...
Cleanup complete
Backup complete
Cleaning up...
Cleanup complete
